In [66]:
# pip install polars


In [67]:
# Notebook to run after q Tues run of after run of cms_api.py outputs upated nh_pre_proc_raw.csv file

# import libraries
import polars as pl
import pandas as pd

import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import table
import s3fs
# import scipy.stats as sts
import numpy as np
# import matplotlib.pyplot as plt
# import sklearn
# import seaborn as sns
#import plotly.express as px
#import plotly.graph_objs as go
#import geopandas as gpd
# import osmnx as ox
import folium
# import geopandas as gpd
# import geopy
# from geopy.geocoders import Nominatim
# import json
# import re
from urllib.request import urlopen
import boto3
from datetime import datetime, timedelta, date
# import os
# import time
# import sys
# import requests
# import sidetable
import warnings
warnings.filterwarnings('ignore')

In [68]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [69]:
m = folium.Map()
m.save("footprint.html")

In [70]:
# #read FIPS dataset
# with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#     counties = json.load(response)

In [71]:
#read FIPS dataset from tsv
fips_df_raw = pd.read_csv("data_pre_proc/fips2county.tsv", sep='\t', header='infer', dtype=str, encoding='latin-1')

In [72]:
fips_df_raw.head(10)

,StateFIPS,CountyFIPS_3,CountyName,StateName,CountyFIPS,StateAbbr,STATE_COUNTY,CountyCBSA
0,01,001,Autauga,Alabama,01001,AL,AL | AUTAUGA,33860
1,01,003,Baldwin,Alabama,01003,AL,AL | BALDWIN,19300
2,01,005,Barbour,Alabama,01005,AL,AL | BARBOUR,21640
3,01,007,Bibb,Alabama,01007,AL,AL | BIBB,13820
4,01,009,Blount,Alabama,01009,AL,AL | BLOUNT,13820
5,01,011,Bullock,Alabama,01011,AL,AL | BULLOCK,NaN
6,01,013,Butler,Alabama,01013,AL,AL | BUTLER,NaN
7,01,015,Calhoun,Alabama,01015,AL,AL | CALHOUN,11500
8,01,017,Chambers,Alabama,01017,AL,AL | CHAMBERS,29300
9,01,019,Cherokee,Alabama,01019,AL,AL | CHEROKEE,NaN


In [73]:
# fips_df.loc[fips_df['StateAbbr']=='LA']

In [74]:
#drop unneeded columns
fips_df = fips_df_raw.drop(columns=["StateFIPS",
                                "CountyFIPS_3",
                                "STATE_COUNTY",
                                "CountyCBSA"])

In [75]:
#read in CMS Nursing Home Covid-19 data from csv
nh_pre_proc_df_raw = pd.read_csv("data_pre_proc/nh_pre_proc.csv")
nh_pre_proc_df_raw.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_total_confirmed_covid_19,residents_weekly_all_deaths,residents_total_all_deaths,residents_weekly_covid_19_deaths,residents_total_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,staff_total_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,total_resident_confirmed_covid_19_cases_per_1_000_residents,total_resident_covid_19_deaths_per_1_000_residents,Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2023-07-16,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,66.0,0.0,48.0,0.0,5.0,57.0,54.0,0.0,0.0,0.0,63.0,0.00,0.0,1222.22,92.59,0.0,54.0,81.0,70.0,NaN,86.420,47.0,NaN,87.037,24.0,NaN,29.630
1,2023-07-16,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,86.0,0.0,5.0,0.0,2.0,85.0,81.0,0.0,0.0,0.0,135.0,0.00,0.0,1061.73,24.69,0.0,81.0,191.0,148.0,NaN,77.487,50.0,NaN,61.728,1.0,NaN,0.524
2,2023-07-16,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,17.0,2.0,42.0,0.0,3.0,50.0,38.0,0.0,0.0,0.0,66.0,0.00,0.0,447.37,78.95,0.0,42.0,84.0,67.0,NaN,79.762,26.0,NaN,61.905,16.0,NaN,19.048
3,2023-07-16,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,107.0,0.0,26.0,0.0,2.0,90.0,78.0,0.0,0.0,0.0,126.0,0.00,0.0,1371.79,25.64,0.0,78.0,92.0,85.0,NaN,92.391,74.0,NaN,94.872,32.0,NaN,34.783
4,2023-07-16,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,75.0,0.0,137.0,0.0,0.0,103.0,84.0,0.0,0.0,0.0,42.0,0.00,0.0,892.86,0.00,0.0,85.0,153.0,127.0,NaN,83.007,59.0,NaN,69.412,8.0,NaN,5.229
5,2023-07-16,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0.0,227.0,0.0,93.0,0.0,0.0,149.0,132.0,0.0,0.0,0.0,248.0,0.00,0.0,1719.70,0.00,0.0,142.0,180.0,101.0,NaN,56.111,69.0,NaN,48.592,26.0,NaN,14.444
6,2023-07-16,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0.0,142.0,1.0,124.0,0.0,7.0,124.0,94.0,0.0,0.0,0.0,176.0,0.00,0.0,1510.64,74.47,0.0,97.0,153.0,146.0,NaN,95.425,58.0,NaN,59.794,36.0,NaN,23.529
7,2023-07-16,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,1.0,161.0,0.0,90.0,0.0,26.0,201.0,82.0,0.0,0.0,2.0,211.0,12.20,0.0,1963.41,317

In [76]:
# #read in CMS Nursing Home Covid-19 data from parquet
# nh_pre_proc_df_raw = pq.read_table("data_pre_proc/nh_pre_proc_raw.parquet")
# nh_pre_proc_df_raw

df = pd.read_parquet("data_pre_proc/nh_pre_proc_raw.parquet", engine='pyarrow')
df.head()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_total_confirmed_covid_19,residents_weekly_all_deaths,residents_total_all_deaths,residents_weekly_covid_19_deaths,residents_total_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,staff_total_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,total_resident_confirmed_covid_19_cases_per_1_000_residents,total_resident_covid_19_deaths_per_1_000_residents,Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2024-01-07,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0,87,1,60,0,5,57,50,0,0,0,70,0,0,1740,100,0,50,81,,,,47,,94,71,,87.654
1,2024-01-07,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0,87,0,5,0,2,85,78,0,0,1,157,0,0,1115.38,25.64,0,80,184,,,,54,,67.5,0,,0
2,2024-01-07,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0,29,0,48,0,3,50,43,0,0,1,86,0,0,674.42,69.77,0,48,87,,,,0,,0,64,,73.563
3,2024-01-07,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0,110,1,35,0,2,90,76,0,0,0,127,0,0,1447.37,26.32,0,76,98,,,,65,,85.526,21,,21.429
4,2024-01-07,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0,94,0,162,0,0,103,83,0,0,0,44,0,0,1132.53,0,0,84,156,,,,59,,70.238,3,,1.923


In [77]:
df.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_total_confirmed_covid_19,residents_weekly_all_deaths,residents_total_all_deaths,residents_weekly_covid_19_deaths,residents_total_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,staff_total_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,total_resident_confirmed_covid_19_cases_per_1_000_residents,total_resident_covid_19_deaths_per_1_000_residents,Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
44704,2024-01-21,745019,JAMES L. WEST ALZHEIMER'S CENTER,1111 SUMMIT AVE,FORT WORTH,TX,76102,8178771199,Tarrant,N,,0,117,0,114,0,14,,,0,0,0,119,,,,,,,,,,,,,,,,
44705,2024-01-21,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,4302602300,Smith,Y,Y,0,39,0,12,0,0,122,91,0,0,0,15,0,0,428.57,0,0,91,139,,,,0,0,0,3,2.158,2.158
44706,2024-01-21,745022,WARE MEMORIAL CARE CENTER,1510 S. VAN BUREN ST.,AMARILLO,TX,79101,8063730471,Potter,Y,Y,0,80,0,154,0,6,120,83,0,0,1,178,0,0,963.86,72.29,0,83,185,,,,45,54.217,54.217,162,87.568,87.568
44707,2024-01-21,745038,TIERRA ESTE NURSING AND REHABILITATION CENTER,14300 PEBBLE HILLS BLVD,EL PASO,TX,79938,9159559998,El Paso,N,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
44708,2024-01-21,745039,MIDTOWNE MEADOWS HEALTH AND REHAB,110 DYLAN WAY,MIDLOTHIAN,TX,76065,0000001111,Ellis,N,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [78]:
# columns in nh_pre_proc_df
list(nh_pre_proc_df_raw.columns.values)

['week_ending',
 'federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'submitted_data',
 'passed_quality_assurance_check',
 'residents_weekly_confirmed_covid_19',
 'residents_total_confirmed_covid_19',
 'residents_weekly_all_deaths',
 'residents_total_all_deaths',
 'residents_weekly_covid_19_deaths',
 'residents_total_covid_19_deaths',
 'number_of_all_beds',
 'total_number_of_occupied_beds',
 'Residents_Hospitalizations_with_Confirmed_COVID_19',
 'Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines',
 'staff_weekly_confirmed_covid_19',
 'staff_total_confirmed_covid_19',
 'weekly_resident_confirmed_covid_19_cases_per_1_000_residents',
 'weekly_resident_covid_19_deaths_per_1_000_residents',
 'total_resident_confirmed_covid_19_cases_per_1_000_residents',
 'total_resident_covid_19_deaths_per_1_000_residents',
 'Number_of_Residents_who_are_Up_to_Date_on_C

In [79]:
# drop unneeded rows from nh pre proc df
nh_pre_proc_df = nh_pre_proc_df_raw.drop(columns=["residents_total_confirmed_covid_19",
                                              "residents_total_all_deaths",
                                              "residents_total_covid_19_deaths",
                                              "staff_total_confirmed_covid_19",
                                              "total_resident_confirmed_covid_19_cases_per_1_000_residents",
                                              "total_resident_covid_19_deaths_per_1_000_residents",
                                              "Number_of_Residents_who_are_Up_to_Date_on_COVID_19_Vaccinations__14_Days_or_More_Before_Positive_Test", 
                                                 ])


In [80]:
# Get rows from fips df where state is not in nh pre proc df
fips_state_notin_nh = fips_df[~fips_df['StateAbbr'].isin(nh_pre_proc_df['provider_state'])]
fips_state_notin_nh

,CountyName,StateName,CountyFIPS,StateAbbr


In [81]:
# Get rows where provider_state values from nh_pre_proc_df are not in fips_df
nh_state_notin_fips = nh_pre_proc_df[~nh_pre_proc_df['provider_state'].isin(fips_df['StateAbbr'])]
nh_state_notin_fips.provider_state.unique()

array(['PR', 'GU'], dtype=object)

In [82]:
# Drop rows from nh_pre_proc_df where provider_state values are not in fips_df
nh_pre_proc_df = nh_pre_proc_df[~nh_pre_proc_df['provider_state'].isin(nh_state_notin_fips['provider_state'])]
nh_pre_proc_df

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines
0,2023-07-16,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,0.0,0.0,0.0,54.0,81.0,70.0,NaN,86.420,47.0,NaN,87.037,24.0,NaN,29.630
1,2023-07-16,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,81.0,0.0,0.0,0.0,0.0,0.0,81.0,191.0,148.0,NaN,77.487,50.0,NaN,61.728,1.0,NaN,0.524
2,2023-07-16,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,2.0,0.0,50.0,38.0,0.0,0.0,0.0,0.0,0.0,42.0,84.0,67.0,NaN,79.762,26.0,NaN,61.905,16.0,NaN,19.048
3,2023-07-16,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,78.0,0.0,0.0,0.0,0.0,0.0,78.0,92.0,85.0,NaN,92.391,74.0,NaN,94.872,32.0,NaN,34.783
4,2023-07-16,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,0.0,0.0,103.0,84.0,0.0,0.0,0.0,0.0,0.0,85.0,153.0,127.0,NaN,83.007,59.0,NaN,69.412,8.0,NaN,5.229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238921,2023-10-29,745007,VERNON REHABILITATION AND NURSING CENTER,4301 HOSPITAL DR.,VERNON,TX,76384,9405522568,Wilbarger,Y,Y,0.0,0.0,0.0,58.0,28.0,0.0,0.0,0.0,0.0,0.0,28.0,54.0,12.0,22.222,22.222,0.0,0.0,0.000,0.0,0.0,0.000
238922,2023-10-29,745017,THE LODGE OF SAGINAW HEALTH AND WELLNESS,848 W MCLEROY BLVD,SAGINAW,TX,76179,6859001210,Tarrant,Y,Y,0.0,2.0,0.0,130.0,90.0,0.0,0.0,0.0,0.0,0.0,97.0,152.0,115.0,75.658,75.658,0.0,0.0,0.000,0.0,0.0,0.000
238923,2023-10-29,745019,JAMES L. WEST ALZHEIMER'S CENTER,1111 SUMMIT AVE,FORT WORTH,TX,76102,8178771199,Tarrant,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
238924,2023-10-29,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,4302602300,Smith,Y,Y,0.0,0.0,0.0,122.0,83.0,0.0,0.0,0.0,0.0,0.0,83.0,95.0,48.0,50.526,50.526,0.0,0.0,0.000,0.0,0.0,0.000


In [83]:
# # keep only alphanumberic chars in column values
# def alphanum(element):
    
#     return "".join(filter(str.isalnum, element))

In [84]:

# nh_pre_proc_df['county_rev'] = [alphanum(x) for x in nh_pre_proc_df['county']]

In [85]:
# nh_pre_proc_df['county_rev']

In [86]:
# fips_df['CountyName_rev'] = [alphanum(x) for x in fips_df['CountyName']]
# fips_df['CountyName_rev']

In [87]:
fips_df.loc[fips_df['StateAbbr']=='LA','CountyName']


1114                  Acadia Parish
1115                   Allen Parish
1116               Ascension Parish
1117              Assumption Parish
1118               Avoyelles Parish
1119              Beauregard Parish
1120               Bienville Parish
1121                 Bossier Parish
1122                   Caddo Parish
1123               Calcasieu Parish
1124                Caldwell Parish
1125                 Cameron Parish
1126               Catahoula Parish
1127               Claiborne Parish
1128               Concordia Parish
1129                 De Soto Parish
1130        East Baton Rouge Parish
1131            East Carroll Parish
1132          East Feliciana Parish
1133              Evangeline Parish
1134                Franklin Parish
1135                   Grant Parish
1136                  Iberia Parish
1137               Iberville Parish
1138                 Jackson Parish
1139               Jefferson Parish
1140         Jefferson Davis Parish
1141               Lafayette

In [88]:
sorted(list(nh_pre_proc_df_raw.loc[nh_pre_proc_df_raw['provider_state']=='LA', 'county'].unique()),reverse=False)


['Acadia',
 'Allen',
 'Ascension',
 'Assumption',
 'Avoyelles',
 'Beauregard',
 'Bienville',
 'Bossier',
 'Caddo',
 'Calcasieu',
 'Caldwell',
 'Catahoula',
 'Claiborne',
 'Concordia',
 'De Soto',
 'East Baton Rouge',
 'East Carroll',
 'East Feliciana',
 'Evangeline',
 'Franklin',
 'Grant',
 'Iberia',
 'Iberville',
 'Jackson',
 'Jefferson',
 'Jefferson Davis',
 'LaSalle',
 'Lafayette',
 'Lafourche',
 'Lincoln',
 'Livingston',
 'Madison',
 'Morehouse',
 'Natchitoches',
 'Orleans',
 'Ouachita',
 'Plaquemines',
 'Pointe Coupee',
 'Rapides',
 'Red River',
 'Richland',
 'Sabine',
 'St. Charles',
 'St. Helena',
 'St. James',
 'St. John the Baptist',
 'St. Landry',
 'St. Martin',
 'St. Mary',
 'St. Tammany',
 'Tangipahoa',
 'Terrebonne',
 'Union',
 'Vermilion',
 'Vernon',
 'Washington',
 'Webster',
 'West Baton Rouge',
 'West Carroll',
 'West Feliciana',
 'Winn']

In [89]:
# Issue: counties in LA have 'Parish' after them in the FIPS dataset, so we will remove 'Parish' from LA county names'
def remove_parish_la(value):
    return value.rstrip(' Parish')


In [90]:
fips_df['CountyName_rev'] = fips_df.apply(lambda row: remove_parish_la(row['CountyName']) if row['StateAbbr'] == 'LA'
                                               else row['CountyName'], axis=1)

In [91]:
# in case any 'Parish' in nh pre proc df, romove those too
nh_pre_proc_df['county_rev'] = nh_pre_proc_df.apply(lambda row: remove_parish_la(row['county']) if row['provider_state'] == 'LA'
                                               else row['county'], axis=1)



In [92]:
# Issue: AK counties don't align between datasets
sorted(list(nh_pre_proc_df.loc[nh_pre_proc_df['provider_state']=='AK','county'].unique()),reverse=False)


['Anchorage',
 'Bethel',
 'Cordova-Mccarthy',
 'Fairbanks',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Matanuska-Susitna',
 'Nome',
 'Not Available',
 'Seward',
 'Sitka',
 'Valdez-Chitina-Whittier',
 'Wrangell-Petersburg']

In [93]:
sorted(list(fips_df.loc[fips_df['StateAbbr']=='AK','CountyName_rev'].unique()),reverse=False)


['Aleutians East Borough',
 'Aleutians West Census Area',
 'Anchorage Municipality',
 'Bethel Census Area',
 'Bristol Bay Borough',
 'Chugach Census Area',
 'Copper River Census Area',
 'Denali Borough',
 'Dillingham Census Area',
 'Fairbanks North Star Borough',
 'Haines Borough',
 'Hoonah-Angoon Census Area',
 'Juneau City and Borough',
 'Kenai Peninsula Borough',
 'Ketchikan Gateway Borough',
 'Kodiak Island Borough',
 'Kusilvak Census Area',
 'Lake and Peninsula Borough',
 'Matanuska-Susitna Borough',
 'Nome Census Area',
 'North Slope Borough',
 'Northwest Arctic Borough',
 'Petersburg Borough',
 'Prince of Wales-Hyder Census Area',
 'Sitka City and Borough',
 'Skagway Municipality',
 'Southeast Fairbanks Census Area',
 'Wrangell City and Borough',
 'Yakutat City and Borough',
 'Yukon-Koyukuk Census Area']

In [94]:
# fips_df['CountyName_rev'] = fips_df.apply(lambda row: remove_ak_counties_ends(row['CountyName_rev']) if row['StateAbbr'] == 'AK'
#                                                else row['CountyName_rev'], axis=1)

In [95]:
fips_df['CountyName_rev']

0          Autauga
1          Baldwin
2          Barbour
3             Bibb
4           Blount
           ...    
3138    Sweetwater
3139         Teton
3140         Uinta
3141      Washakie
3142        Weston
Name: CountyName_rev, Length: 3143, dtype: object

In [96]:
fips_df['CountyName_rev']=np.where(fips_df['StateAbbr']=='AK', fips_df['CountyName_rev'].str.replace(' City and Borough| North Star Borough| Municipality| Census Area| Gateway Borough','',regex=True),fips_df['CountyName_rev'])



In [97]:
# Issue: County name diff in AK
    
fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Kenai Peninsula Borough': 'Kenai-Cook Inlet'}).where(fips_df['StateAbbr']=='AK', fips_df['CountyName_rev'])


In [98]:
sorted(list(nh_pre_proc_df.loc[nh_pre_proc_df['provider_state']=='AK','county_rev'].unique()),reverse=False)

['Anchorage',
 'Bethel',
 'Cordova-Mccarthy',
 'Fairbanks',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Matanuska-Susitna',
 'Nome',
 'Not Available',
 'Seward',
 'Sitka',
 'Valdez-Chitina-Whittier',
 'Wrangell-Petersburg']

In [99]:
sorted(list(fips_df.loc[fips_df['StateAbbr']=='AK','CountyName_rev'].unique()),reverse=False)

['Aleutians East Borough',
 'Aleutians West',
 'Anchorage',
 'Bethel',
 'Bristol Bay Borough',
 'Chugach',
 'Copper River',
 'Denali Borough',
 'Dillingham',
 'Fairbanks',
 'Haines Borough',
 'Hoonah-Angoon',
 'Juneau',
 'Kenai-Cook Inlet',
 'Ketchikan',
 'Kodiak Island Borough',
 'Kusilvak',
 'Lake and Peninsula Borough',
 'Matanuska-Susitna Borough',
 'Nome',
 'North Slope Borough',
 'Northwest Arctic Borough',
 'Petersburg Borough',
 'Prince of Wales-Hyder',
 'Sitka',
 'Skagway',
 'Southeast Fairbanks',
 'Wrangell',
 'Yakutat',
 'Yukon-Koyukuk']

In [100]:
# Issue: differences in VA counties
# Revise CountyName_rev in fips df to match those in nh pre proc df


fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Albemarle': 'Albermarle',  'Bedford':'Bedford City',
 'Poquoson city':'Poquoson'}).where(fips_df['StateAbbr']=='VA', fips_df['CountyName_rev'])

In [101]:
# Issue: non alpha characters in NM county

fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'DoÃ\x83Â±a Ana': 'Dona Ana'}).where(fips_df['StateAbbr']=='NM', fips_df['CountyName_rev'])


In [102]:
# Issue: Spelling diff in NH county

fips_df['CountyName_rev'] = fips_df['CountyName_rev'].replace({'Hillsborough':'Hillsboro'}).where(fips_df['StateAbbr']=='NH', fips_df['CountyName_rev'])


In [103]:
nh_pre_proc_df['state_county'] = (nh_pre_proc_df['provider_state'].fillna('')
                                  + nh_pre_proc_df['county_rev'].fillna('')).astype(str).str.lower()



fips_df['state_county'] = (fips_df['StateAbbr'].fillna('') + fips_df['CountyName_rev'].fillna('')).astype(str).str.lower()

# nh_pre_proc_df_reset = nh_pre_proc_df.reset_index(drop=True)
# fips_df_reset = fips_df.reset_index(drop=True)

In [104]:
# Send fips_df to aws S3 as parquet

# connect to AWS s3
s3_client = boto3.client('s3')

DESTINATION = 'fips-codes-smw'

def write_df_to_parquet_s3(dataframe,filename):
    print("Writing {} records to {}".format(len(dataframe),filename))
    output_file = f"s3://{DESTINATION}/{filename}/data.parquet"
    dataframe.to_parquet(output_file)


write_df_to_parquet_s3(fips_df,'fips-codes')

Writing 3143 records to fips-codes


In [105]:
s3 = s3fs.S3FileSystem()

fips_codes_df = pq.ParquetDataset(
    's3://fips-codes-smw/fips-codes/data.parquet',
    filesystem=s3,
).read_pandas().to_pandas()

fips_codes_df.head()

,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev,state_county
0,Autauga,Alabama,01001,AL,Autauga,alautauga
1,Baldwin,Alabama,01003,AL,Baldwin,albaldwin
2,Barbour,Alabama,01005,AL,Barbour,albarbour
3,Bibb,Alabama,01007,AL,Bibb,albibb
4,Blount,Alabama,01009,AL,Blount,alblount


In [40]:
#which are in fips_df_sc but not in nh_pre_proc_df_sc?

not_in_nh=list(set(fips_df['state_county']) - set(nh_pre_proc_df['state_county']))
not_in_nh_sorted = sorted(not_in_nh, reverse=False)
not_in_nh_sorted

['akaleutians east borough',
 'akaleutians west',
 'akbristol bay borough',
 'akchugach',
 'akcopper river',
 'akdenali borough',
 'akdillingham',
 'akhaines borough',
 'akhoonah-angoon',
 'akkusilvak',
 'aklake and peninsula borough',
 'akmatanuska-susitna borough',
 'aknorth slope borough',
 'aknorthwest arctic borough',
 'akpetersburg borough',
 'akprince of wales-hyder',
 'akskagway',
 'aksoutheast fairbanks',
 'akwrangell',
 'akyakutat',
 'akyukon-koyukuk',
 'arcalhoun',
 'ardesha',
 'azgreenlee',
 'azla paz',
 'azsanta cruz',
 'caalpine',
 'camono',
 'casierra',
 'coclear creek',
 'cocostilla',
 'cocuster',
 'codolores',
 'coelbert',
 'cogilpin',
 'cogrand',
 'cohinsdale',
 'cojackson',
 'cokiowa',
 'colake',
 'comineral',
 'coouray',
 'copark',
 'copitkin',
 'cosaguache',
 'cosan juan',
 'cosan miguel',
 'cosummit',
 'flglades',
 'flliberty',
 'flunion',
 'gaatkinson',
 'gabaker',
 'gabanks',
 'gachattahoochee',
 'gadawson',
 'gadooly',
 'gaechols',
 'galincoln',
 'gamcintosh',


In [41]:
#which are in nh_pre_proc_df_sc but not in fips_df_sc?

not_in_fips=list(set(nh_pre_proc_df['state_county']) - set(fips_df['state_county']))
not_in_fips_sorted = sorted(not_in_fips, reverse=False)
not_in_fips_sorted

# WIP 5 state_counties in AK in fips data that have no direct match in nh pre proc data: 
# 'akcordova-mccarthy',
#  'akmatanuska-susitna',
#  'aknot available',
#  'akseward',
#  'akvaldez-chitina-whittier',
#  'akwrangell-petersburg'

['akcordova-mccarthy',
 'akmatanuska-susitna',
 'aknot available',
 'akseward',
 'akvaldez-chitina-whittier',
 'akwrangell-petersburg']

In [42]:
#merge nh_pre_proc_df and fips df on state_county
nh_pre_proc_fips_merge = nh_pre_proc_df.merge(fips_df, how='left', left_on=["state_county"], right_on=["state_county"])


In [43]:
nh_pre_proc_fips_merge.head(10)

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
0,2023-07-16,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Y,Y,0.0,0.0,0.0,57.0,54.0,0.0,0.0,0.0,0.00,0.0,54.0,81.0,70.0,NaN,86.420,47.0,NaN,87.037,24.0,NaN,29.630,Franklin,alfranklin,Franklin,Alabama,01059,AL,Franklin
1,2023-07-16,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Y,Y,0.0,0.0,0.0,85.0,81.0,0.0,0.0,0.0,0.00,0.0,81.0,191.0,148.0,NaN,77.487,50.0,NaN,61.728,1.0,NaN,0.524,Talladega,altalladega,Talladega,Alabama,01121,AL,Talladega
2,2023-07-16,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Y,Y,0.0,2.0,0.0,50.0,38.0,0.0,0.0,0.0,0.00,0.0,42.0,84.0,67.0,NaN,79.762,26.0,NaN,61.905,16.0,NaN,19.048,Jackson,aljackson,Jackson,Alabama,01071,AL,Jackson
3,2023-07-16,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Y,Y,0.0,0.0,0.0,90.0,78.0,0.0,0.0,0.0,0.00,0.0,78.0,92.0,85.0,NaN,92.391,74.0,NaN,94.872,32.0,NaN,34.783,Jefferson,aljefferson,Jefferson,Alabama,01073,AL,Jefferson
4,2023-07-16,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Y,Y,0.0,0.0,0.0,103.0,84.0,0.0,0.0,0.0,0.00,0.0,85.0,153.0,127.0,NaN,83.007,59.0,NaN,69.412,8.0,NaN,5.229,Jefferson,aljefferson,Jefferson,Alabama,01073,AL,Jefferson
5,2023-07-16,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,Limestone,Y,Y,0.0,0.0,0.0,149.0,132.0,0.0,0.0,0.0,0.00,0.0,142.0,180.0,101.0,NaN,56.111,69.0,NaN,48.592,26.0,NaN,14.444,Limestone,allimestone,Limestone,Alabama,01083,AL,Limestone
6,2023-07-16,015019,MERRY WOOD LODGE,280 MT HEBRON ROAD,ELMORE,AL,36025,3345678484,Elmore,Y,Y,0.0,1.0,0.0,124.0,94.0,0.0,0.0,0.0,0.00,0.0,97.0,153.0,146.0,NaN,95.425,58.0,NaN,59.794,36.0,NaN,23.529,Elmore,alelmore,Elmore,Alabama,01051,AL,Elmore
7,2023-07-16,015023,HATLEY HEALTH CARE INC,300 MEDICAL CENTER DRIVE,CLANTON,AL,35045,2057554960,Chilton,Y,Y,1.0,0.0,0.0,201.0,82.0,0.0,0.0,2.0,12.20,0.0,82.0,133.0,112.0,NaN,84.211,49.0,NaN,59.756,3.0,NaN,2.256,Chilton,alchilton,Chilton,Alabama,01021,AL,Chilton
8,2023-07-16,015024,"LIMESTONE NURSING AND REHABILITATI

In [44]:
nh_pre_proc_fips_merge.tail()

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
238809,2023-10-29,745007,VERNON REHABILITATION AND NURSING CENTER,4301 HOSPITAL DR.,VERNON,TX,76384,9405522568,Wilbarger,Y,Y,0.0,0.0,0.0,58.0,28.0,0.0,0.0,0.0,0.0,0.0,28.0,54.0,12.0,22.222,22.222,0.0,0.0,0.0,0.0,0.0,0.0,Wilbarger,txwilbarger,Wilbarger,Texas,48487,TX,Wilbarger
238810,2023-10-29,745017,THE LODGE OF SAGINAW HEALTH AND WELLNESS,848 W MCLEROY BLVD,SAGINAW,TX,76179,6859001210,Tarrant,Y,Y,0.0,2.0,0.0,130.0,90.0,0.0,0.0,0.0,0.0,0.0,97.0,152.0,115.0,75.658,75.658,0.0,0.0,0.0,0.0,0.0,0.0,Tarrant,txtarrant,Tarrant,Texas,48439,TX,Tarrant
238811,2023-10-29,745019,JAMES L. WEST ALZHEIMER'S CENTER,1111 SUMMIT AVE,FORT WORTH,TX,76102,8178771199,Tarrant,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tarrant,txtarrant,Tarrant,Texas,48439,TX,Tarrant
238812,2023-10-29,745021,LINDALE SPECIALTY CARE CENTER,13905 FM 2710,LINDALE,TX,75771,4302602300,Smith,Y,Y,0.0,0.0,0.0,122.0,83.0,0.0,0.0,0.0,0.0,0.0,83.0,95.0,48.0,50.526,50.526,0.0,0.0,0.0,0.0,0.0,0.0,Smith,txsmith,Smith,Texas,48423,TX,Smith
238813,2023-10-29,745022,WARE MEMORIAL CARE CENTER,1510 S. VAN BUREN ST.,AMARILLO,TX,79101,8063730471,Potter,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Potter,txpotter,Potter,Texas,48375,TX,Potter


In [45]:
# Is the row count of nh_pre_proc_fips_merge the same as nh_pre_proc_df?
if len(nh_pre_proc_fips_merge.index) == len(nh_pre_proc_df.index):
    print(len(nh_pre_proc_fips_merge.index))
else:
    print(f'nh_pre_proc_fips_merge and nh_pre_proc_df do not have equal row counts. \n nh_pre_proc_fips_merge: {len(nh_pre_proc_fips_merge.index)},\n nh_pre_proc_df: {len(nh_pre_proc_df.index)}')
    

238814


In [46]:
# Convert week_ending to datetime
nh_pre_proc_fips_merge['week_ending'] = pd.to_datetime(nh_pre_proc_fips_merge['week_ending'])

In [47]:
# nh_pre_proc_fips_merge.loc[(nh_pre_proc_fips_merge['week_ending'] == nh_pre_proc_fips_merge['week_ending'].max()) 
#                    & (nh_pre_proc_fips_merge['federal_provider_number'] == '015010')]

In [48]:
# new_cols = ['CountyName','CountyFIPS', 'StateAbbr', 'STATE_COUNTY']

# How many fips codes are NULL?
null_rows = nh_pre_proc_fips_merge.loc[nh_pre_proc_fips_merge['CountyFIPS'].isnull()]
null_rows

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev
226,2023-07-16,025015,WRANGELL MEDICAL CENTER LTC,232 WOOD STREET,WRANGELL,AK,99929,9078747000,Wrangell-Petersburg,Y,Y,0.0,0.0,0.0,18.0,12.0,0.0,0.0,0.0,0.0,0.0,12.0,48.0,44.0,NaN,91.667,10.0,NaN,83.333,12.0,NaN,25.000,Wrangell-Petersburg,akwrangell-petersburg,NaN,NaN,NaN,NaN,NaN
228,2023-07-16,025019,PETERSBURG MEDICAL CENTER LTC,103 FRAM STREET,PETERSBURG,AK,99833,9077724291,Wrangell-Petersburg,Y,Y,0.0,0.0,0.0,15.0,13.0,0.0,0.0,1.0,0.0,0.0,13.0,92.0,90.0,NaN,97.826,8.0,NaN,61.538,21.0,NaN,22.826,Wrangell-Petersburg,akwrangell-petersburg,NaN,NaN,NaN,NaN,NaN
231,2023-07-16,025024,PROVIDENCE SEWARD MOUNTAIN HAVEN,2203 OAK STREET,SEWARD,AK,99664,9072245241,Seward,Y,Y,0.0,0.0,0.0,40.0,40.0,0.0,0.0,0.0,0.0,0.0,40.0,99.0,93.0,NaN,93.939,31.0,NaN,77.500,8.0,NaN,8.081,Seward,akseward,NaN,NaN,NaN,NaN,NaN
235,2023-07-16,025028,CORDOVA COMMUNITY MED LTC,602 CHASE AVE,CORDOVA,AK,99574,9074248000,Cordova-Mccarthy,Y,Y,1.0,1.0,1.0,10.0,8.0,1.0,0.0,0.0,125.0,125.0,8.0,28.0,27.0,NaN,96.429,6.0,NaN,75.000,9.0,NaN,32.143,Cordova-Mccarthy,akcordova-mccarthy,NaN,NaN,NaN,NaN,NaN
239,2023-07-16,025034,PROVIDENCE VALDEZ MEDICAL CENTER,911 MEALS AVENUE,VALDEZ,AK,99686,9078341825,Valdez-Chitina-Whittier,Y,Y,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,61.0,45.0,NaN,73.770,9.0,NaN,90.000,6.0,NaN,9.836,Valdez-Chitina-Whittier,akvaldez-chitina-whittier,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224132,2023-10-29,025028,CORDOVA COMMUNITY MED LTC,602 CHASE AVE,CORDOVA,AK,99574,9074248000,Cordova-Mccarthy,Y,Y,0.0,0.0,0.0,10.0,9.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,96.429,NaN,NaN,11.111,NaN,NaN,0.000,NaN,Cordova-Mccarthy,akcordova-mccarthy,NaN,NaN,NaN,NaN,NaN
224136,2023-10-29,025034,PROVIDENCE VALDEZ MEDICAL CENTER,911 MEALS AVENUE,VALDEZ,AK,99686,9078341825,Valdez-Chitina-Whittier,Y,Y,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,0.0,10.0,64.0,47.0,73.438,73.438,7.0,70.000,70.000,0.0,0.000,0.000,Valdez-Chitina-Whittier,akvaldez-chitina-whittier,NaN,NaN,NaN,NaN,NaN
224137,2023-10-29,025035,UTUQQANAAT INAAT,436 MISSION STREET,KOTZEBUE,AK,99752,9074423321,Not Available,Y,Y,0.0,0.0,0.0,18.0,17.0,0.0,0.0

In [49]:
# sorted(list(null_rows.loc[null_rows['provider_state']=='AK','county_rev'].unique()),reverse=False)

# Are the fips null rows all for the state_county values that we cannot match on AK?
sorted(list(null_rows['state_county'].unique()),reverse=False)


['akcordova-mccarthy',
 'akmatanuska-susitna',
 'aknot available',
 'akseward',
 'akvaldez-chitina-whittier',
 'akwrangell-petersburg']

In [50]:
# What is the date range of week_ending?
max_date = max(nh_pre_proc_fips_merge['week_ending'])
max_date

Timestamp('2023-10-29 00:00:00')

In [51]:
min_date = min(nh_pre_proc_fips_merge['week_ending'])
min_date

Timestamp('2023-07-16 00:00:00')

In [52]:
duplicate_rows = nh_pre_proc_fips_merge.duplicated()
nh_pre_proc_fips_merge[duplicate_rows]

,week_ending,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,submitted_data,passed_quality_assurance_check,residents_weekly_confirmed_covid_19,residents_weekly_all_deaths,residents_weekly_covid_19_deaths,number_of_all_beds,total_number_of_occupied_beds,Residents_Hospitalizations_with_Confirmed_COVID_19,Residents_Hospitalizations_with_Confirmed_COVID_19_and_Up_to_Date_with_Vaccines,staff_weekly_confirmed_covid_19,weekly_resident_confirmed_covid_19_cases_per_1_000_residents,weekly_resident_covid_19_deaths_per_1_000_residents,number_of_residents_staying_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week,number_of_all_healthcare_personnel_eligible_to_work_in_this_facility_for_at_least_1_day_this_week_who_received_a_completed_covid_19_vaccination_at_any_time,recent_percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,percentage_of_current_healthcare_personnel_who_received_a_completed_covid_19_vaccination_at_any_time,Number_of_Residents_Staying_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Residents_Up_to_Date_with_COVID_19_Vaccines,Number_of_Healthcare_Personnel_Eligible_to_Work_in_this_Facility_for_At_Least_1_Day_This_Week_Up_to_Date_with_COVID_19_Vaccines,Recent_Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,Percentage_of_Current_Healthcare_Personnel_Up_to_Date_with_COVID_19_Vaccines,county_rev,state_county,CountyName,StateName,CountyFIPS,StateAbbr,CountyName_rev


In [53]:
# Extract unique nursing home information

nursing_homes = nh_pre_proc_fips_merge[['federal_provider_number','provider_name','provider_address',
                                       'provider_city','provider_state','provider_zip_code','provider_phone_number','county',
                                       'county_rev','state_county','CountyFIPS','CountyName_rev']]

nursing_homes = nursing_homes.drop_duplicates().reset_index(drop=True)

# Add a unique ID for each nursing home
nursing_homes['nursing_home_id'] = range(1, len(nursing_homes) + 1)


In [54]:
nursing_homes.columns.to_list()

['federal_provider_number',
 'provider_name',
 'provider_address',
 'provider_city',
 'provider_state',
 'provider_zip_code',
 'provider_phone_number',
 'county',
 'county_rev',
 'state_county',
 'CountyFIPS',
 'CountyName_rev',
 'nursing_home_id']

In [55]:
nursing_homes.head()

,federal_provider_number,provider_name,provider_address,provider_city,provider_state,provider_zip_code,provider_phone_number,county,county_rev,state_county,CountyFIPS,CountyName_rev,nursing_home_id
0,015009,"BURNS NURSING HOME, INC.",701 MONROE STREET NW,RUSSELLVILLE,AL,35653,2563324110,Franklin,Franklin,alfranklin,01059,Franklin,1
1,015010,COOSA VALLEY HEALTHCARE CENTER,260 WEST WALNUT STREET,SYLACAUGA,AL,35150,2562495604,Talladega,Talladega,altalladega,01121,Talladega,2
2,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,Jackson,Jackson,aljackson,01071,Jackson,3
3,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,Jefferson,Jefferson,aljefferson,01073,Jefferson,4
4,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY,MC CALLA,AL,35111,2054776161,Jefferson,Jefferson,aljefferson,01073,Jefferson,5


### Send to S3

In [55]:
# connect to AWS s3
s3_client = boto3.client('s3')

# if neccessary to use a client operation, uncomment the following and add the client operation
# s3_resource.meta.client.<client_operation>

In [ ]:
# df.to_parquet('df.parquet.gzip',
#               compression='gzip')

In [58]:
# s3_url = 's3://bucket/folder/bucket.parquet.gzip'
# df.to_parquet(s3_url, compression='gzip')


DESTINATION = 'cms-nh-fac-wkd3610641-87b4-43e5-81cb-4b68255f58a6'

def write_df_to_parquet_s3(dataframe,filename):
    print("Writing {} records to {}".format(len(dataframe),filename))
    output_file = f"s3://{DESTINATION}/{filename}/data.parquet"
    dataframe.to_parquet(output_file)


write_df_to_parquet_s3(nursing_homes,'cms-nh-fac-wk')


# s3_url = 's3://cms-nh-fac-wkd3610641-87b4-43e5-81cb-4b68255f58a6/cms-nh-fac-wk/cms-nh-fac-wkd3610641-87b4-43e5-81cb-4b68255f58a6.parquet.gzip'
# nursing_homes.to_parquet(s3_url, compression = 'gzip')

Writing 14942 records to cms-nh-fac-wk


### Preparing data for Polars

In [60]:
# # Preparing data for Polars

# #define variables for date range
# start_date = '2021-07-04'
# end_date = '2022-07-03'

# #convert week_ending to datetime
# nh_latest_sub_2['week_ending'] = pd.to_datetime(nh_latest_sub_2['week_ending'])

# # create week start date col (should be a Mon)
# nh_latest_sub_2['week_start'] = nh_latest_sub_2['week_ending'] - pd.Timedelta(days = 6)

# nh_latest_sub_2['month_year'] = pd.to_datetime(nh_latest_sub_2['week_start']).dt.to_period('M')

# # converting period to string for ease of manipulation with polars
# nh_latest_sub_2['month_year'] = nh_latest_sub_2['month_year'].dt.strftime('%Y-%m')

# #create a boolean mask for rows within the date range
# dr_mask = (nh_latest_sub_2['week_ending'] >= start_date) & (nh_latest_sub_2['week_ending'] <= end_date)

# # Apply the boolean mask to subset the DataFrame
# dr_nh_latest_sub_2 = nh_latest_sub_2[dr_mask]


In [61]:
# For each state_county, what are the nh's that passed qa and how many times in the date range?

### WIP Testing with previous data

### WIP Send to RDS tables

In [62]:
# import pandas as pd
# import pymysql
# from sqlalchemy import create_engine

In [63]:

# Split processed data into tables

In [64]:
# def find_location(row):
    
#     place = row['provider_name']
    
#     location = geolocator.geocode(place)
    
#     if location != None:
#         return location.address, location.latitude, location.longitude, location.raw['importance']
#     else:
#         return "Not Found", "Not Found", "Not Found", "Not Found"

In [65]:
# nh_pre_proc_df[['address', 'lat', 'lon', 'importance']] = nh_pre_proc_df.apply(find_location, axis="columns", result_type="expand")
# nh_pre_proc_df.head(10)